In [1]:
import sys
sys.path.append('..')
from configure.settings import DBSelector

In [15]:
conn = DBSelector().get_mysql_conn('ptrade','qq',use_dict=True)

In [3]:
import pandas as pd

In [ ]:
df = pd.read_sql('uqerio',con=engine)

In [16]:
cursor = conn.cursor()

In [17]:
sql='select tickerBond,tradeDate,closePriceBond from uqerio where `tradeDate`>=%s'
cursor.execute(sql,('2021-11-01',))
ret=cursor.fetchall()

In [18]:
len(ret)

96608

In [19]:
df=pd.DataFrame(ret)

In [20]:
df.head()

,tickerBond,tradeDate,closePriceBond
0,110033,2021-11-01,102.89
1,110033,2021-11-02,102.10
2,110033,2021-11-03,101.86
3,110033,2021-11-04,101.64
4,110034,2021-11-01,110.03


In [31]:
df['tickerBond']=df['tickerBond'].astype(str)

In [24]:
last_bond_df = pd.read_excel('../data/lastBond.xlsx')

In [25]:
last_bond_df.head()

,code,name,price,lastDate
0,128073,哈尔转债,109.999,2022-10-27
1,123027,蓝晓转债,403.133,2022-10-18
2,113635,升21转债,118.324,2022-09-23
3,123137,锦浪转债,155.000,2022-09-21
4,118001,金博转债,108.794,2022-09-20


In [39]:
name_mapper = dict(zip(last_bond_df['code'].tolist(),last_bond_df['name'].tolist()))

In [33]:
last_bond_df['code']=last_bond_df['code'].astype(str)

In [26]:
len(last_bond_df)

42

In [34]:
last_bond_code_list = last_bond_df['code'].tolist()

In [35]:
last_df = df[df['tickerBond'].isin(last_bond_code_list)]

In [36]:
last_df.head()

,tickerBond,tradeDate,closePriceBond
1616,113502,2021-11-01,123.55
1617,113502,2021-11-02,122.56
1618,113502,2021-11-03,122.78
1619,113502,2021-11-04,124.64
1620,113502,2021-11-05,128.65


In [32]:
last_bond_code_list[0]

128073

In [37]:
result_dict ={}
for code,sub_df in last_df.groupby('tickerBond'):
    tmp_df=sub_df.sort_values('tradeDate')
    t1=tmp_df['closePriceBond'].iloc[-1]
    t2=tmp_df['closePriceBond'].iloc[-22]
    p = (t1-t2)/t2*100
    result_dict[code]=p


In [38]:
result_dict

{'110066': -15.511818100116825,
 '110071': 20.513753456556564,
 '113026': 2.1709955001184067,
 '113034': -15.82927071933142,
 '113036': -2.6247762974746522,
 '113502': -28.019942060230406,
 '113541': 4.831075567161744,
 '113548': -5.594631981402143,
 '113550': -12.213697073032101,
 '113568': 7.229664410992956,
 '113599': 9.828730134238551,
 '113620': 0.6559455491792177,
 '113630': -5.968427881496433,
 '113635': -15.45205753524499,
 '118001': -17.35490732300212,
 '118002': -31.333561878642435,
 '123023': -29.60352710646636,
 '123024': -17.39130434782608,
 '123027': -0.7061576354679849,
 '123042': -21.214095194499958,
 '123043': -9.761772853185597,
 '123045': -20.501237380681157,
 '123062': -31.945708219593577,
 '123070': 24.08957842481523,
 '123073': -15.530177678566059,
 '123084': 4.406779661016949,
 '123086': -4.276150772371082,
 '123097': -24.305850635174135,
 '123111': -19.649122807017545,
 '123137': -18.9961797553162,
 '127011': -22.437673130193904,
 '127013': 7.232586129080795,
 '

In [50]:
new_result_dict = {}
for k,v in result_dict.items():
    new_result_dict[name_mapper.get(k)]=round(v,3)

In [51]:
import numpy as np
np.mean(list(new_result_dict.values()))

-8.48390476190476

In [52]:
new_result_dict

{'盛屯转债': -15.512,
 '湖盐转债': 20.514,
 '核能转债': 2.171,
 '滨化转债': -15.829,
 '宁建转债': -2.625,
 '嘉澳转债': -28.02,
 '荣晟转债': 4.831,
 '石英转债': -5.595,
 '常汽转债': -12.214,
 '新春转债': 7.23,
 '嘉友转债': 9.829,
 '傲农转债': 0.656,
 '赛伍转债': -5.968,
 '升21转债': -15.452,
 '金博转债': -17.355,
 '天合转债': -31.334,
 '迪森转债': -29.604,
 '岱勒转债': -17.391,
 '蓝晓转债': -0.706,
 '银河转债': -21.214,
 '正元转债': -9.762,
 '雷迪转债': -20.501,
 '三超转债': -31.946,
 '鹏辉转债': 24.09,
 '同和转债': -15.53,
 '高澜转债': 4.407,
 '海兰转债': -4.276,
 '美力转债': -24.306,
 '东财转3': -19.649,
 '锦浪转债': -18.996,
 '中鼎转2': -22.438,
 '创维转债': 7.233,
 '中大转债': -22.715,
 '钧达转债': -3.146,
 '哈尔转债': -17.892,
 '百川转债': -0.452,
 '星帅转债': -14.155,
 '奥瑞转债': -13.378,
 '同德转债': -7.785,
 '交科转债': -0.143,
 '比音转债': -6.919,
 '祥鑫转债': 35.523}

In [67]:
from pyecharts import options as opts
from pyecharts.charts import Bar,Line,Grid
from pyecharts.commons.utils import JsCode
import pandas as pd

import datetime
today = datetime.date.today().strftime('%Y%m%d')
color_function = """
        function (params) {
            if(params.value>0){
            return 'red';
            }else{
                return 'green';
            }
        }
        """
title='可转债 退市前一个月的涨跌幅分布'
x=list(new_result_dict.keys())
y=list(new_result_dict.values())
c = (
    Bar()
    .add_xaxis(
x    
    )
    .add_yaxis(title,y,itemstyle_opts=opts.ItemStyleOpts(color=JsCode(color_function)),
    label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=90),
        min_interval=1,
        interval=1,
        splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
         yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True)),
        title_opts=opts.TitleOpts(title=title),
    ).set_colors(['green'])
    .render("../plot_image/{}{}.html".format(title,today))
)

In [68]:
new_result_dict

{'盛屯转债': -15.512,
 '湖盐转债': 20.514,
 '核能转债': 2.171,
 '滨化转债': -15.829,
 '宁建转债': -2.625,
 '嘉澳转债': -28.02,
 '荣晟转债': 4.831,
 '石英转债': -5.595,
 '常汽转债': -12.214,
 '新春转债': 7.23,
 '嘉友转债': 9.829,
 '傲农转债': 0.656,
 '赛伍转债': -5.968,
 '升21转债': -15.452,
 '金博转债': -17.355,
 '天合转债': -31.334,
 '迪森转债': -29.604,
 '岱勒转债': -17.391,
 '蓝晓转债': -0.706,
 '银河转债': -21.214,
 '正元转债': -9.762,
 '雷迪转债': -20.501,
 '三超转债': -31.946,
 '鹏辉转债': 24.09,
 '同和转债': -15.53,
 '高澜转债': 4.407,
 '海兰转债': -4.276,
 '美力转债': -24.306,
 '东财转3': -19.649,
 '锦浪转债': -18.996,
 '中鼎转2': -22.438,
 '创维转债': 7.233,
 '中大转债': -22.715,
 '钧达转债': -3.146,
 '哈尔转债': -17.892,
 '百川转债': -0.452,
 '星帅转债': -14.155,
 '奥瑞转债': -13.378,
 '同德转债': -7.785,
 '交科转债': -0.143,
 '比音转债': -6.919,
 '祥鑫转债': 35.523}